In [1]:
import warnings 
warnings.filterwarnings('ignore')

### Importing Libraries

In [3]:
# Importing necessary dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from scipy import stats

### Datasets

In [4]:
#Datasets
products= pd.read_csv('../data/products.csv')
orders= pd.read_csv('../data/orders.csv')
order_products= pd.read_csv('../data/order_products.csv')
aisles= pd.read_csv('../data/aisles.csv')

**The above datasets are imported in order to preprocess them according to the dataset description previously made.**

In [5]:
# Displaying the first 5 rows of the dataset
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


#### Key Insights:
* orders dataset contains a column that has 'NaN' values present in the column 'days_since_prior_order'.
* This could mean that there are some missing values in the column of the dataset.

In [6]:
# Checking null values for each column
orders.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

#### Key Insights:
* All features of this dataset has no missing values except for 'days_since_prior_order'.
* 'days_since_prior_order' contains a total of 206209 missing values
* Dropping the rows of the dataset could bring the huge drawback to the analysis process. This is because dropping the dataset could potentially lose potential information from the dataset.

In [7]:
# Replace NaN values with '-1'
orders['days_since_prior_order'] = orders['days_since_prior_order'].replace(np.nan, -1)

**The missing values in the feature 'days_since_prior_order' has been replaced by '-1' to ensure that the those data are utilized. This value indicates that the user has not purchased any items previously and this is their first time.**

In [8]:
# Adding/Incrementing the values in the column with 1
orders['days_since_prior_order']= orders['days_since_prior_order'].add(1)

**1 is added to the values of the feature to remove negative value that exists due to the replacement made. This ensures smoothe analysis of the data.**

In [9]:
# Displaying the first 5 rows of the dataset
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,0.0
1,2398795,1,prior,2,3,7,16.0
2,473747,1,prior,3,3,12,22.0
3,2254736,1,prior,4,4,7,30.0
4,431534,1,prior,5,4,15,29.0


**Missing values are removed and thus replaced with 0. Other values are incremented with 1. This keeps the values of this column within the positive value range.**

In [10]:
# dropping the column that is not required
orders= orders.drop(['eval_set'], axis=1)

# Displaying the first 5 rows of the dataset
orders.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,0.0
1,2398795,1,2,3,7,16.0
2,473747,1,3,3,12,22.0
3,2254736,1,4,4,7,30.0
4,431534,1,5,4,15,29.0


The column which held no significance to the data (eval_set) in general has been removed.

In [11]:
# Checking null values for each column
orders.isna().sum()

order_id                  0
user_id                   0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
dtype: int64

The dataset now has no missing values. This means that the missing values have been successfully treated.

In [12]:
# Saving the dataset in desired directory
orders.to_csv('../results/orders_processed.csv', index=False)

**The processed dataset is saved and stored in the desired location.**

### Merged Dataset

In [13]:
# Merge order_products with products to get product details in each order
order_products_details = pd.merge(order_products, products, on='product_id', how='left')

# Merge the result with orders to get user and order details
dataset = pd.merge(order_products_details, orders, on='order_id', how='left')

dataset= pd.merge(dataset, aisles, on= 'aisle_id', how='left')

# Displaying the first 5 rows of the dataset
dataset.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle
0,2,33120,1,1,Organic Egg Whites,86,16,202279,3,5,9,9.0,eggs
1,2,28985,2,1,Michigan Organic Kale,83,4,202279,3,5,9,9.0,fresh vegetables
2,2,9327,3,0,Garlic Powder,104,13,202279,3,5,9,9.0,spices seasonings
3,2,45918,4,1,Coconut Butter,19,13,202279,3,5,9,9.0,oils vinegars
4,2,30035,5,0,Natural Sweetener,17,13,202279,3,5,9,9.0,baking ingredients


In [14]:
# Writing the column names in order in a list
new_order= ['order_id', 'product_id', 'aisle_id', 
             'department_id', 'user_id', 'aisle', 
             'product_name', 'order_number', 'order_dow', 
             'order_hour_of_day', 'days_since_prior_order', 
             'add_to_cart_order',	'reordered']
dataset= dataset[new_order]

# Displaying the first 5 rows of the dataset
dataset.head()

,order_id,product_id,aisle_id,department_id,user_id,aisle,product_name,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
0,2,33120,86,16,202279,eggs,Organic Egg Whites,3,5,9,9.0,1,1
1,2,28985,83,4,202279,fresh vegetables,Michigan Organic Kale,3,5,9,9.0,2,1
2,2,9327,104,13,202279,spices seasonings,Garlic Powder,3,5,9,9.0,3,0
3,2,45918,19,13,202279,oils vinegars,Coconut Butter,3,5,9,9.0,4,1
4,2,30035,17,13,202279,baking ingredients,Natural Sweetener,3,5,9,9.0,5,0


**The four datasets 'products.csv', 'orders.csv', 'order_products.csv' and 'aisles.csv' are merged using left joing. The order of the columns are rearranged for a more refined understanding of the dataset.** 

In [15]:
# Determining the number of rows and columns of the dataset
dataset.shape

(32434489, 13)

The merged dataset has a total of 32434489 rows and 13 columns, indicating a fairly large dataset.

In [16]:
# Determining the names of the columns of the dataset
dataset.columns.tolist()

['order_id',
 'product_id',
 'aisle_id',
 'department_id',
 'user_id',
 'aisle',
 'product_name',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order',
 'add_to_cart_order',
 'reordered']

The merged dataset contains all the columns of the four datasets.

In [17]:
# dataset information
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 13 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   product_id              int64  
 2   aisle_id                int64  
 3   department_id           int64  
 4   user_id                 int64  
 5   aisle                   object 
 6   product_name            object 
 7   order_number            int64  
 8   order_dow               int64  
 9   order_hour_of_day       int64  
 10  days_since_prior_order  float64
 11  add_to_cart_order       int64  
 12  reordered               int64  
dtypes: float64(1), int64(10), object(2)
memory usage: 3.1+ GB


### Data Insights:
* **order_id:** Integer (int64) - Unique identifier for each order.
* **product_id:** Integer (int64) - Unique identifier for each product.
* **aisle_id:** Integer (int64) - Represents the aisle location, treated as categorical.
* **department_id:** Integer (int64) - Represents product departments, treated as categorical.
* **user_id:** Integer (int64) - Unique identifier for users.
* **aisle:** Object (string) - Name of the aisle, corresponds to aisle_id.
* **product_name:** Object (string) - Product names, not used for categorical analysis.
* **order_number:** Integer (int64) - Order sequence number, indicating the position of the order.
* **order_dow:** Integer (int64) - Day of the week (1-7), treated as categorical.
* **order_hour_of_day:** Integer (int64) - Hour of the day (0-23), treated as categorical.
* **days_since_prior_order:** Float (float64) - Days since the last order, with some missing values.
* **add_to_cart_order:** Integer (int64) - Product's position in the cart, treated as categorical.
* **reordered:** Integer (int64) - Binary feature indicating if the product was reordered (1) or not (0).

In [18]:
# Checking null values for each column
dataset.isna().sum()

order_id                  0
product_id                0
aisle_id                  0
department_id             0
user_id                   0
aisle                     0
product_name              0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
add_to_cart_order         0
reordered                 0
dtype: int64

**There are no missing values in the dataset.**

In [19]:
# Saving the dataset in desired directory
dataset.to_csv('../results/dataset_processed.csv', index=False)

## Encoding

### Data Stratification

In [20]:
def stratified_split(dataset, test_size=0.99, random_state=42):
    """
    This function splits the dataset into training and testing sets with stratified sampling
    based on the 'user_id' column. It also prints the size of the training and testing sets 
    as well as the distribution of 'user_id' in both sets.
    
    Parameters:
    - dataset: The input DataFrame to be split
    - test_size: Proportion of the dataset to include in the test split (default is 0.99)
    - random_state: The random seed for reproducibility (default is 42)
    
    Returns:
    - train_data: The stratified training set
    - test_data: The stratified testing set
    """
    
    # Split the dataset into train and test sets with stratification based on 'user_id'
    train_data, test_data = train_test_split(
        dataset, 
        test_size=test_size, 
        stratify=dataset['user_id'], 
        random_state=random_state
    )
    
    # Display the size of the training and testing sets
    print(f"Training set size: {train_data.shape[0]} rows")
    print(f"Testing set size: {test_data.shape[0]} rows")
    
    # Display the distribution of user_id in both sets
    print("\nUser ID distribution in training set:")
    print(train_data['user_id'].value_counts(normalize=True))
    
    print("\nUser ID distribution in testing set:")
    print(test_data['user_id'].value_counts(normalize=True))
    
    return train_data, test_data

# Example usage:
train_data, test_data = stratified_split(dataset)

Training set size: 324344 rows
Testing set size: 32110145 rows

User ID distribution in training set:
user_id
201268    0.000114
129928    0.000111
164055    0.000096
33731     0.000089
176478    0.000089
            ...   
77853     0.000003
92756     0.000003
100822    0.000003
120602    0.000003
37156     0.000003
Name: proportion, Length: 143811, dtype: float64

User ID distribution in testing set:
user_id
201268    1.148547e-04
129928    1.121764e-04
164055    9.436270e-05
186704    9.053214e-05
176478    9.006499e-05
              ...     
98654     9.342842e-08
22226     9.342842e-08
170850    9.342842e-08
204913    9.342842e-08
49581     9.342842e-08
Name: proportion, Length: 206209, dtype: float64


### Key Insights:
* **Training Set Size:** 324,344 rows.
* **Testing Set Size:** 32,110,145 rows, much larger than the training set.
#### User Distribution in Training Set:
* User IDs are distributed with varying proportions.
* Top users (e.g., user_id 201268) have a very small proportion (0.000114).
* A large number of users have extremely low proportions (e.g., user_id 37156 at 0.000003).
#### User Distribution in Testing Set:
* Similar pattern to the training set, with top users (e.g., user_id 201268) having small proportions.
* Many users in the testing set have proportions close to 0 (e.g., user_id 49581 at 9.34e-08).

In [21]:
# Displaying the first 5 rows of the dataset
train_data.head()

,order_id,product_id,aisle_id,department_id,user_id,aisle,product_name,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
2045015,215678,2664,92,18,74160,baby food formula,First Prunes,18,2,12,12.0,7,0
13212535,1394471,18471,29,13,21299,honeys syrups nectars,Organic Light Agave Nectar,10,2,16,4.0,5,0
25783757,2719214,35515,105,13,166931,doughs gelatins bake mixes,Peanut Butter Cookie Mix,1,6,11,0.0,1,0
31007500,3270713,17862,81,15,145193,canned jarred vegetables,Minced Garlic,38,1,8,6.0,11,1
11043095,1165817,13415,98,7,197157,juice nectars,100% Pear Juice,10,0,23,24.0,15,1


In [22]:
# Determining the number of rows and columns of the dataset
train_data.shape

(324344, 13)

#### Target Encoding on Categorical Variables with Datatype Object

In [23]:
train_data['aisle_target_enc']        = train_data['aisle'].map(train_data.groupby('aisle')['reordered'].mean())
train_data['product_name_target_enc'] = train_data['product_name'].map(train_data.groupby('product_name')['reordered'].mean())

# Displaying the first 5 rows of the dataset
train_data.head()

,order_id,product_id,aisle_id,department_id,user_id,aisle,product_name,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,aisle_target_enc,product_name_target_enc
2045015,215678,2664,92,18,74160,baby food formula,First Prunes,18,2,12,12.0,7,0,0.591705,0.620690
13212535,1394471,18471,29,13,21299,honeys syrups nectars,Organic Light Agave Nectar,10,2,16,4.0,5,0,0.383502,0.351351
25783757,2719214,35515,105,13,166931,doughs gelatins bake mixes,Peanut Butter Cookie Mix,1,6,11,0.0,1,0,0.424214,0.333333
31007500,3270713,17862,81,15,145193,canned jarred vegetables,Minced Garlic,38,1,8,6.0,11,1,0.420020,0.290909
11043095,1165817,13415,98,7,197157,juice nectars,100% Pear Juice,10,0,23,24.0,15,1,0.598418,0.333333


### Insights on Target Encoding:
##### Target Encoding Process:

* **aisle_target_enc:** This column represents the average reorder probability (i.e., the mean of the "reordered" column) for each aisle.
* **product_name_target_enc:** This column captures the average reorder probability for each product_name.
* Target encoding transforms categorical variables (aisle, product_name) into numerical features, reflecting the likelihood that a product in a specific aisle or with a specific name will be reordered.

##### Insights from Target Encoding:

* Higher encoded values (e.g., product_name_target_enc = 0.620690 for "First Prunes") indicate products and aisles with a higher probability of being reordered.
* Lower values (e.g., aisle_target_enc = 0.383502 for "honeys syrups nectars") indicate areas with lower reorder probabilities.
* The encoded columns effectively capture reorder trends for aisles and products, which can be valuable for predictive modeling, especially for demand forecasting or customer retention.

#### Apply Binning on day_sin_prior_order

In [24]:
train_data.days_since_prior_order

2045015     12.0
13212535     4.0
25783757     0.0
31007500     6.0
11043095    24.0
            ... 
24834069    13.0
20836554     0.0
5788339     31.0
4514209      3.0
10973074     3.0
Name: days_since_prior_order, Length: 324344, dtype: float64

In [25]:
bins = [-1, 6, 12, 18, 24, float('inf')]
labels = ['undefined', 'recently_ordered', 'moderately_ordered', 'infrequently_ordered', 'least_ordered']
train_data['encoded_days_since_prior_order']= pd.cut(train_data['days_since_prior_order'], bins=bins, labels=labels)

**The days_since_prior_order' feature is binned and categorized into 'undefined', 'recently_ordered', 'infrequently_ordered' and 'least_ordered'. This helps in grouping together rows that fall in the same category.**

In [26]:
train_data.to_csv('../results/encoded_data.csv')

In [27]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 324344 entries, 2045015 to 10973074
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype   
---  ------                          --------------   -----   
 0   order_id                        324344 non-null  int64   
 1   product_id                      324344 non-null  int64   
 2   aisle_id                        324344 non-null  int64   
 3   department_id                   324344 non-null  int64   
 4   user_id                         324344 non-null  int64   
 5   aisle                           324344 non-null  object  
 6   product_name                    324344 non-null  object  
 7   order_number                    324344 non-null  int64   
 8   order_dow                       324344 non-null  int64   
 9   order_hour_of_day               324344 non-null  int64   
 10  days_since_prior_order          324344 non-null  float64 
 11  add_to_cart_order               324344 non-null  int64   
 12 

In [28]:
# Displaying the first 5 rows of the dataset
train_data.head()

,order_id,product_id,aisle_id,department_id,user_id,aisle,product_name,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,aisle_target_enc,product_name_target_enc,encoded_days_since_prior_order
2045015,215678,2664,92,18,74160,baby food formula,First Prunes,18,2,12,12.0,7,0,0.591705,0.620690,recently_ordered
13212535,1394471,18471,29,13,21299,honeys syrups nectars,Organic Light Agave Nectar,10,2,16,4.0,5,0,0.383502,0.351351,undefined
25783757,2719214,35515,105,13,166931,doughs gelatins bake mixes,Peanut Butter Cookie Mix,1,6,11,0.0,1,0,0.424214,0.333333,undefined
31007500,3270713,17862,81,15,145193,canned jarred vegetables,Minced Garlic,38,1,8,6.0,11,1,0.420020,0.290909,undefined
11043095,1165817,13415,98,7,197157,juice nectars,100% Pear Juice,10,0,23,24.0,15,1,0.598418,0.333333,infrequently_ordered


### The datasets are preprocessed